```{contents}
```
## Directed Graph 

### **1. Formal Definition**

In **LangGraph**, a **Directed Graph (DG)** is the core execution structure that defines:

> **how control flows between computational units (nodes) using directed edges under shared state.**

Formally, a LangGraph graph is a tuple:

$$
G = (V, E, S)
$$

Where:

* **V** → Set of **Nodes**
* **E** → Set of **Directed Edges**
  $e = (u \rightarrow v)$
* **S** → **State schema** shared across all nodes

---

### **2. Why Directed Graphs in LangGraph?**

Traditional LLM pipelines are linear:

```
A → B → C
```

LangGraph generalizes this into:

```
A → B → C
↑   ↓   ↑
D ← E ← F
```

This enables:

* **Branching**
* **Loops**
* **Conditional execution**
* **Parallelism**
* **Human-in-the-loop**
* **Multi-agent orchestration**

All essential for **production-grade LLM systems**.

---

### **3. Core Components**

| Component         | Role                                                            |
| ----------------- | --------------------------------------------------------------- |
| **Node**          | Computation unit (LLM call, tool, function, agent, human input) |
| **Directed Edge** | Allowed transition between nodes                                |
| **State**         | Shared mutable memory                                           |
| **Entry Point**   | First node executed                                             |
| **END**           | Terminal node                                                   |

---

### **4. Execution Semantics**

At runtime, LangGraph behaves as a **state machine**:

1. Start at **entry node**
2. Execute node logic on current **state**
3. Emit **partial state update**
4. Follow outgoing **directed edges**
5. Continue until reaching **END**

**Edges encode control flow.
State encodes data flow.**

---

### **5. Types of Directed Edges**

| Edge Type            | Purpose                 |
| -------------------- | ----------------------- |
| **Static Edge**      | Always transitions      |
| **Conditional Edge** | Branches based on state |
| **Cyclic Edge**      | Forms loops             |
| **Dynamic Edge**     | Computed at runtime     |
| **Parallel Edge**    | Fan-out execution       |

---

### **6. Example: Simple Directed Graph**



In [1]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

class State(TypedDict):
    text: str
    result: str

def process(state: State):
    return {"result": state["text"].upper()}

builder = StateGraph(State)

builder.add_node("process", process)
builder.set_entry_point("process")
builder.add_edge("process", END)

graph = builder.compile()

print(graph.invoke({"text": "langgraph"}))


{'text': 'langgraph', 'result': 'LANGGRAPH'}




**Graph structure:**

```
START → process → END
```

---

### **7. Example: Conditional Directed Graph**



In [2]:
def router(state):
    return "long" if len(state["text"]) > 5 else "short"

def long_handler(state):
    return {"result": f"Long text: {state['text']}"}

def short_handler(state):
    return {"result": f"Short text: {state['text']}"}

builder.add_node("router", lambda s: s)
builder.add_node("long", long_handler)
builder.add_node("short", short_handler)

builder.add_conditional_edges("router", router, {
    "long": "long",
    "short": "short"
})


Adding a node to a graph that has already been compiled. This will not be reflected in the compiled graph.
Adding a node to a graph that has already been compiled. This will not be reflected in the compiled graph.
Adding a node to a graph that has already been compiled. This will not be reflected in the compiled graph.
Adding an edge to a graph that has already been compiled. This will not be reflected in the compiled graph.




**Graph structure:**

```
router → long
      → short
```

---

### **8. Directed Graph vs Traditional Pipeline**

| Feature       | Pipeline | LangGraph Directed Graph |
| ------------- | -------- | ------------------------ |
| Flow          | Linear   | Arbitrary topology       |
| Loops         | ❌        | ✅                        |
| Branching     | ❌        | ✅                        |
| Memory        | Local    | Global state             |
| Resumability  | ❌        | ✅                        |
| Human Control | ❌        | ✅                        |
| Multi-Agent   | ❌        | ✅                        |

---

### **9. Design Intuition**

LangGraph uses directed graphs because:

> **LLM systems are not functions — they are evolving decision processes.**

A directed graph naturally models:

* decision points
* feedback loops
* coordination
* supervision
* recovery

---

### **10. Common Graph Patterns**

| Pattern          | Shape                           |
| ---------------- | ------------------------------- |
| **Linear Chain** | A → B → C                       |
| **Branching**    | A → {B, C}                      |
| **Loop**         | A → B → A                       |
| **Supervisor**   | S → {A, B, C} → S               |
| **Map-Reduce**   | Split → Parallel → Join         |
| **ReAct Loop**   | Reason → Act → Observe → Reason |

---

### **11. Mental Model**

Think of LangGraph’s directed graph as:

> **A programmable decision system with memory.**

Nodes make decisions.
Edges define legal transitions.
State carries knowledge forward.

